# Encrypting SB40 file using KDP (Key Derivation Provider)

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)

## 2. Key Derivation plugin Setup

First, we need to setup the Key Derivation Plugin (KDP) and start a custom HSM. To do that please go to [Key Derivation Plugin](../_common/sb_key_derivation_provider.ipynb) and follow the steps there. Once completed, you can continue with this example.

## 3. Preparing configuration file

For demonstration purposes will be creating an SB4.0 file for mcxn556s. There's nothing special about mcxn556s, this example will work exactly the same for every device which support SB3.1 or SB4.0.

There's a prepared [configuration file](sb40_config.yaml) for this specific example.

Let's examine portions of the configuration relevant for encryption (and usage of Key Derivation Provider):

```yaml
containerKeyBlobEncryptionKey: type=mysbkdp;key_id=0
```

Normally, this setting would point to a CUST_MK_SK (PCK) file located on the computer.

Here, the `type` refers to a concrete Key Derivation plugin. This identifier is used to select a specific encryption provider. 
(For reference see line #15 [demo plugin implementation](../_common/plugins/spsdk_sbkdp/spsdk_sbkdp/provider.py))

The `key_id` is just a internal reference to a key stored in the HSM.

## 4. Generating the SB file

In [6]:
# This env variable sets colored logger output to STDOUT
%env JUPYTER_SPSDK=1
# Set a magic for command execution and echo
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [7]:
! nxpimage -v sb40 export -c sb40_config.yaml

INFO:spsdk.utils.service_provider:Loading plugins: spsdk.sb31kdp
INFO:spsdk.utils.plugins:Plugin mysbkdp-spsdk.sb31kdp has been loaded.
INFO:spsdk.sbfile.sb31.images:Adding list (1): [SB3.1 Command: CmdErase]
INFO:spsdk.sbfile.sb31.images:Adding list (1): [SB3.1 Command: CmdLoad]
SRKH: 412600fd846385bd8263770692fba46721d30c12aede0dc19cbaf54c5c473948179953bdaf43eb2a3cfdbfde7ae8f17bdcbb3ba79b675c2b5746c41c1f4d2d6b
Success. (Secure binary 4.0: output/plugin.sb4 created.)


In the command's output you may notice the following details:
```
INFO:spsdk.utils.service_provider:Loading plugins: spsdk.sb31kdp
INFO:spsdk.utils.plugins:Plugin mysbkdp-spsdk.sb31kdp has been loaded.
```
SPSDK loads relevant installed plugins before attempting to preform any operation that could be delegated to a plugin.

## 5. Parsing the generated SB file

For parsing we'll use `nxpimage sb40 parse` command. This command normally expects the used CUST_MK_SK (PCK) key to be provided on the command line. However, we may used the same configuration string as used during generation (`type=mysbkdp;key_id=0`).

In [ ]:
! nxpimage -v sb40 parse -f mcxn556s -b output/plugin.sb4 -k "type=mysbkdp;key_id=0" -o parsed

INFO:spsdk.utils.service_provider:Loading plugins: spsdk.sb31kdp
INFO:spsdk.utils.plugins:Plugin mysbkdp-spsdk.sb31kdp has been loaded.
Success. (SB4.0: output/plugin.sb4 has been parsed and stored into parsed/sb4_mcxn646_config.yaml.)


As a quick check, we may compare the input binary provided in the config file with the decrypted output to verify integrity and correctness of the decryption process.

In [9]:
import filecmp

result = filecmp.cmp("inputs/my_binary.bin", "parsed/load_data_00000000.bin")
print(f"Binary files match: {result}")

Binary files match: True
